In [7]:
#expand skills list with dutch skills 
import pandas as pd
df_tags=pd.read_csv("tags.csv")
SKILLS_DB = df_tags.name.str.lower().tolist()
SKILLS_DB_DICT ={}
for x in SKILLS_DB:
    SKILLS_DB_DICT[x] = x
SKILLS_DB_DICT["programma manager"] = "program manager"
SKILLS_DB_DICT["zelfstandig"] = "independent"
SKILLS_DB_DICT["autonoom"] = "independent"
SKILLS_DB_DICT["nederlands"] = "dutch"
SKILLS_DB_DICT["engels"] = "english"
SKILLS_DB_DICT["frans"] = "french"
SKILLS_DB_DICT["duits"] = "german"
SKILLS_DB_DICT["accountant"] = "auditor"
SKILLS_DB_DICT["applicatiebeheerder"] = "application administrator"
SKILLS_DB_DICT["logistiek"] = "logistics"
SKILLS_DB_DICT["digitale transformatie"] = "digital transformation"
SKILLS_DB_DICT["functioneel analist"] = "functional analyst"
SKILLS_DB_DICT["business analist"] = "business analyst"
SKILLS_DB_DICT["project ingenieur"] = "project engineer"
SKILLS_DB_DICT["data analist"] = "data analyst"
SKILLS_DB_DICT["data science"] = "data scientist"
SKILLS_DB_DICT["analist"] = "analyst"
SKILLS_DB_DICT["strategie"] = "strategy"
SKILLS_DB_DICT["qlik"] = "qlik (view/sense)"
SKILLS_DB_DICT["automatisatie"] = "automation"
SKILLS_DB_DICT["packaging engineer"] = "packaging engineer/specialist"
SKILLS_DB_DICT["packaging specialist"] = "packaging engineer/specialist"

In [8]:
SKILLS_DB_DICT

{'coo': 'coo',
 'external party mngmt': 'external party mngmt',
 'it architect': 'it architect',
 '.net': '.net',
 'symfony': 'symfony',
 'android': 'android',
 'green field': 'green field',
 'cobol': 'cobol',
 'treasury': 'treasury',
 'front end developer': 'front end developer',
 'telecom': 'telecom',
 'crm': 'crm',
 'audit': 'audit',
 'program manager': 'program manager',
 'business developer': 'business developer',
 'life science': 'life science',
 'angular': 'angular',
 'independent': 'independent',
 'it delivery': 'it delivery',
 'lean six sigma': 'lean six sigma',
 'django': 'django',
 'dutch': 'dutch',
 'english': 'english',
 'french': 'french',
 'german': 'german',
 'student': 'student',
 'auditor': 'auditor',
 'application adminstrator': 'application adminstrator',
 'data scientist': 'data scientist',
 'machine learning': 'machine learning',
 'itil': 'itil',
 'data manager': 'data manager',
 'network security': 'network security',
 'digital transformation manager': 'digital t

In [2]:
from pdfminer.high_level import extract_text
import docx2txt
import nltk
import requests
import os
import sys
import pandas as pd
import re
import subprocess
df_tags=pd.read_csv("tags.csv")

SKILLS_DB = df_tags.name.str.lower().tolist()
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')


nltk.download('stopwords')

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None

def preprocessing_text(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text.lower())
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
    
    # remove the stop words
    filtered_tokens = [w for w in filtered_tokens if w not in stop_words]

    return filtered_tokens

def extract_skills(input_text):
    if input_text == ["error"]:
        found_skills = "error"
    else:
        input_text = preprocessing_text(input_text)
        # generate bigrams and trigrams (such as artificial intelligence)
        bigrams_trigrams = list(map(' '.join, nltk.everygrams(input_text, 2, 3)))

        # we create a set to keep the results in.
        found_skills = set()

        # we search for each token in our skills database
        for token in input_text:
            if token.lower() in SKILLS_DB_DICT.keys():
                found_skills.add(SKILLS_DB_DICT[token])

        # we search for each bigram and trigram in our skills database
        for ngram in bigrams_trigrams:
            if ngram.lower() in SKILLS_DB_DICT.keys():
                found_skills.add(SKILLS_DB_DICT[ngram.lower()])

    return found_skills

def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names[0],person_names[1]

def extract_names_NER(text):
    nlp = spacy.load("nl_core_news_sm")
    doc = nlp(text)
    start =[]
    stop=[]
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            start.append(ent.start_char)
            stop.append(ent.end_char)
            #print(ent.text,ent.start_char,ent.end_char)
    name = text[start[0]:stop[0]]
    first_name = name.split(" ")[0]
    try:
        last_name = name.split(" ")[1]
    except:
        last_name = text[start[1]:stop[1]]
    return first_name,last_name

def extract_phone_number(resume_text):
    if resume_text == ["error"]:
        found_number = "error"
    else:
    
        phone = re.findall(PHONE_REG, resume_text)
        list =[]
        list2 =[]
        for i in phone:
            list.append(i.replace(' ', '').replace('.','') .replace('+','').replace(')','').replace('(','').replace('+','').replace('-',''))
        for i in list:
            if len(i)>8:
                list2.append(i)
        if list2:
            found_number = ''.join(list2[0])
        else: found_number = "nan"

    return found_number

EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_emails(resume_text):
    if resume_text == ["error"]:
        emails = ["error"]
    else:
        emails = re.findall(EMAIL_REG, resume_text.lower())
        if emails:
            return emails[0]


import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    if resume_text == ["error"]:
        first_name,last_name = ["error","error"]
    else:
        nlp_text = nlp(resume_text)

        # First name and Last name are always Proper Nouns
        pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

        matcher.add('NAME',[pattern])

        matches = matcher(nlp_text)

        if nlp_text[matches[0][1]:matches[0][2]] is not "curriculum vitae":
            span = nlp_text[matches[0][1]:matches[0][2]]
        else:
            span = nlp_text[matches[1][1]:matches[1][2]]

        first_name = span[0]
        last_name = span[1]
    return first_name,last_name

def parse_id(candidate_id):
    try:
        dirs = os.listdir(sys.path[0]+'/candidates/'+candidate_id)
    except:
        print("map not opening for candidate: {}".format(candidate_id))
        dirs = []
    try:
        a = dirs[0]
        path = sys.path[0]+'/candidates/'+candidate_id+'/'+a
    except:
        print("map empty for candidate: {}".format(candidate_id))
    
    if len(dirs) > 0:
        
        # This would print all the files and directories
        # This would print all the files and directories
        try:
            with open(path, 'rb') as f:            
                if path[-4:] == "docx":
                    text = extract_text_from_docx(f)
                elif path[-4:] == ".pdf":
                    text = extract_text(f)
                else:
                    text = ['error']
        except:
            text =["error"]
        try:
            skills = extract_skills(text)
        except:
            print("skills not working for candidate: {}".format(candidate_id))
            skills ='error'
        try:
            number = extract_phone_number(text)
        except:
            print("number not working for candidate: {}".format(candidate_id))
            number ='error'
        try:
            mail = extract_emails(text)
        except:
            print("number not working for candidate: {}".format(candidate_id))
            mail = 'error'
        try:
            first_name,last_name = extract_name(text)
        except:
            print("extract names not working for candidate: {}".format(candidate_id))
            first_name,last_name = ['error','error']
        try:
            first_name2,last_name2 = extract_names(text)
        except:
            print("extract names 2 not working for candidate: {}".format(candidate_id))
            first_name2,last_name2 = ['error','error']      
        try:
            first_name3,last_name3 = extract_names_NER(text)
        except:
            print("extract names 3 not working for candidate: {}".format(candidate_id))
            first_name3,last_name3 = ['error','error']
        
        

    else:
        skills,number,mail, first_name, last_name,first_name2,last_name2,first_name3,last_name3 = ['nan','nan','nan','nan','nan','nan','nan','nan','nan']
    return skills, number, mail, first_name, last_name,first_name2,last_name2,first_name3,last_name3 


def parse_id_lbk(candidate_id):
    try:
        dirs = os.listdir(sys.path[0]+'/candidates/'+candidate_id+'/candidates/scripts/LBK/')
    except:
        print("map not opening for candidate: {}".format(candidate_id))
        dirs = []
    try:
        a = dirs[0]
        path = sys.path[0]+'/candidates/'+candidate_id+'/candidates/scripts/LBK/'+a
    except:
        print("map empty for candidate: {}".format(candidate_id))
    
    if len(dirs) > 0:
        
        # This would print all the files and directories
        # This would print all the files and directories
        try:
            with open(path, 'rb') as f:            
                if path[-4:] == "docx":
                    text = extract_text_from_docx(f)
                elif path[-4:] == ".pdf":
                    text = extract_text(f)
                   
                else:
                    text = ['error']
        except:
            text =["error"]
        try:
            skills = extract_skills(text)
        except:
            print("skills not working for candidate: {}".format(candidate_id))
            skills ='error'
        try:
            number = extract_phone_number(text)
        except:
            print("number not working for candidate: {}".format(candidate_id))
            number ='error'
        try:
            mail = extract_emails(text)
        except:
            print("number not working for candidate: {}".format(candidate_id))
            mail = 'error'
        try:
            first_name,last_name = extract_name(text)
        except:
            print("extract names not working for candidate: {}".format(candidate_id))
            first_name,last_name = ['error','error']
        try:
            first_name2,last_name2 = extract_names(text)
        except:
            print("extract names 2 not working for candidate: {}".format(candidate_id))
            first_name2,last_name2 = ['error','error']      
        try:
            first_name3,last_name3 = extract_names_NER(text)
        except:
            print("extract names 3 not working for candidate: {}".format(candidate_id))
            first_name3,last_name3 = ['error','error']       

    else:
        skills,number,mail, first_name, last_name,first_name2,last_name2,first_name3,last_name3 = ['nan','nan','nan','nan','nan','nan','nan','nan','nan']
    return skills, number, mail, first_name, last_name,first_name2,last_name2,first_name3,last_name3 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JefRutten\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
"""helaas kan de data omtrent CV's en de nimbus database niet gedeeld worden, om toch een POC demonstratie te kunnen delen.
deel ik 2 CV's waarvan ik toestemming heb om deze te delen, deze kunnen gebruikt worden tijdens de trial"""

"helaas kan de data omtrent CV's en de nimbus database niet gedeeld worden, om toch een POC demonstratie te kunnen delen.\ndeel ik 2 CV's waarvan ik toestemming heb om deze te delen, deze kunnen gebruikt worden tijdens de trial"

In [4]:
"""maak een map aan candidates, vervolgens een map jef en plaats hier de CV"""

'maak een map aan candidates, vervolgens een map jef en plaats hier de CV'

In [5]:
parse_id('jef')

({'dutch', 'engineering', 'english', 'french', 'logistics'},
 '32494062706',
 'jefrutten@hotmail.com',
 CURRICULUM,
 VITAE,
 'Leuven',
 'België Major',
 'Minor',
 'Skikot')

In [ ]:
"""results:
we zien dat de tags, tel nr en e-mail succesvol zijn geëxtraheerd
we zien voor first name en last name dat er fouten zijn gemaakt in alle gebruikte methodes POS pattern mathcing, NER nltk en NER spacy"""

In [ ]:
"""voeg in de map candidates een map yanne toe, vervolgens plaats de CV in deze map"""

In [9]:
parse_id('yanne')

({'bi',
  'dutch',
  'english',
  'fmcg',
  'french',
  'management',
  'retail',
  'student'},
 '32487527728',
 'yannebillen@hotmail.com',
 Yanne,
 Billen,
 'Yanne',
 'Billen WORK',
 'Yanne',
 'Billen')

In [ ]:
"""results:
we zien dat de tags, tel nr en e-mail succesvol zijn geëxtraheerd
ook de naam is succesvol bij POS pattern matching en NER door spacy
bij NER nltk is er iets misgelopen bij de achternaam"""